## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [1]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek

### 1st-Example
Let's first construct a small, 5-bus grid ourselves to really understand each step.

#### Define Variables and admittance matrix

In [2]:
n = 5  # Number of buses
G = [0, 1]  # Generator buses are either on or off
N = range(n)  # All buses
Y = np.random.randn(n, n) + 1j * np.random.randn(n, n)  # Admittance matrix
c2 = np.array([0.1, 0.2])  # Quadratic cost coefficients
c1 = np.array([10, 20])  # Linear cost coefficients
c0 = np.array([100, 200])  # Constant cost coefficients
P_min = np.array([0, 0])  # Minimum active power
P_max = np.array([100, 150])  # Maximum active power
Q_min = np.array([-50, -50])  # Minimum reactive power
Q_max = np.array([50, 50])  # Maximum reactive power
V_min = 0.95  # Minimum voltage magnitude
V_max = 1.05  # Maximum voltage magnitude
S_max = 200  # Maximum apparent power flow

#### Define Optimization Variables
Let's define the power flow constraint variables (P, Q) and the PSD matrix (X).

In [3]:
# Define optimization variables
W = cp.Variable((n, n), hermitian=True)  # X = VV^+
P = cp.Variable(len(G))  # Active power generation
Q = cp.Variable(len(G))  # Reactive power generation

#### Define Objective function

In [4]:
# Define objective
objective = cp.Minimize(cp.sum(c2 @ cp.square(P) + c1 @ P + c0))

#### Constraints

In [5]:
constraints = [W >> 0]  # X is positive semidefinite
for i in G:
    constraints += [P_min[i] <= P[i], P[i] <= P_max[i]]
    constraints += [Q_min[i] <= Q[i], Q[i] <= Q_max[i]]
for i in N:
    constraints += [V_min**2 <= cp.real(W[i, i]), cp.real(W[i, i]) <= V_max**2]
for i in N:
    for j in N:
        if i != j:
            S_ij = Y[i, j].conj() * W[i, j]
            constraints += [cp.abs(S_ij) <= S_max]

#### Solve the problem using cvxpy's SDP solvers. (probably IPM)

In [6]:
problem = cp.Problem(objective, constraints)
# Solve using a free solver (e.g., ECOS)
problem.solve()

# Print results
print("Optimal value:", problem.value)
print("X matrix:", W.value)
print("P:", P.value)
print("Q:", Q.value)

Optimal value: 300.0000000001779
X matrix: [[1.00235207+0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 1.00235207+0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 1.00235207+0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 1.00235207+0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  1.00235207+0.j]]
P: [2.82054517e-12 3.03767099e-12]
Q: [-50. -50.]


/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [7]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    else:
        raise ValueError("Unsupported test case")

In [8]:
# Load the chosen IEEE test case
case_number = 30 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)
pp.runpp(net)  # Run power flow to initialize values
net

This pandapower network includes the following parameter tables:
   - bus (30 elements)
   - load (20 elements)
   - gen (5 elements)
   - shunt (2 elements)
   - ext_grid (1 element)
   - line (41 elements)
   - poly_cost (6 elements)
   - bus_geodata (30 elements)
 and the following results tables:
   - res_bus (30 elements)
   - res_line (41 elements)
   - res_ext_grid (1 element)
   - res_load (20 elements)
   - res_shunt (2 elements)
   - res_gen (5 elements)

In [9]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,2.00,0.02000,0.0,0.0,0.0
1,0,gen,0.0,1.75,0.01750,0.0,0.0,0.0
2,1,gen,0.0,1.00,0.06250,0.0,0.0,0.0
3,2,gen,0.0,3.25,0.00834,0.0,0.0,0.0
4,3,gen,0.0,3.00,0.02500,0.0,0.0,0.0
5,4,gen,0.0,3.00,0.02500,0.0,0.0,0.0


When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [10]:
#Gneration bus information
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,max_p_mw,min_p_mw
0,None,1,60.97,1.0,NaN,-20.0,60.0,1.0,False,True,0.0,None,True,80.0,0.0
1,None,21,21.59,1.0,NaN,-15.0,62.5,1.0,False,True,0.0,None,True,50.0,0.0
2,None,26,26.91,1.0,NaN,-15.0,48.7,1.0,False,True,0.0,None,True,55.0,0.0
3,None,22,19.20,1.0,NaN,-10.0,40.0,1.0,False,True,0.0,None,True,30.0,0.0
4,None,12,37.00,1.0,NaN,-15.0,44.7,1.0,False,True,0.0,None,True,40.0,0.0


In [11]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,None,1,21.7,12.7,0.0,0.0,NaN,1.0,True,None,False
1,None,2,2.4,1.2,0.0,0.0,NaN,1.0,True,None,False
2,None,3,7.6,1.6,0.0,0.0,NaN,1.0,True,None,False
3,None,6,22.8,10.9,0.0,0.0,NaN,1.0,True,None,False
4,None,7,30.0,30.0,0.0,0.0,NaN,1.0,True,None,False
5,None,9,5.8,2.0,0.0,0.0,NaN,1.0,True,None,False
6,None,11,11.2,7.5,0.0,0.0,NaN,1.0,True,None,False
7,None,13,6.2,1.6,0.0,0.0,NaN,1.0,True,None,False
8,None,14,8.2,2.5,0.0,0.0,NaN,1.0,True,None,False
9,None,15,3.5,1.8,0.0,0.0,NaN,1.0,True,None,False


In [32]:
# Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (we need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus
branch = net.line  # Get each transmission line's specification for line constraints

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #index of the generators.
P_min_gen = net.gen["min_p_mw"].values #minimal real power generation limit
P_max_gen = net.gen["max_p_mw"].values #maximum real power generation limit
Q_min_gen = net.gen["min_q_mvar"].values #minimal reactive power generation limit
Q_max_gen = net.gen["max_q_mvar"].values #maximum real power generation limit

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw'] #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']

# Voltage limits
V_min = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
V_max = net.bus["max_vm_pu"].values ** 2

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values  # Quadratic cost coefficients
c1 = cost["cp1_eur_per_mw"].values   # Linear cost coefficients
c0 = cost["cp0_eur"].values          # Constant cost coefficients

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")

### Define Variable
Define the SDP Matrix X in CVXPY. We want X to be PSD. If X is rank-1, then our solution is the global optimal. In most power system cases, X is rank-1.
Represents voltage relations in SDP relaxation.

In [33]:
# Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
W = cp.Variable((n, n), hermitian=True)

# Define phi and psi matrices for each bus as the constraints
Phi = []
Psi = []
for j in range(n):
    Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
    Y_j[j,:] = Ybus[j,:]
    Y_j_H = np.conj(Y_j).T #Hermitian of Y_j

    Phi_j = 1/2 * (Y_j_H + Y_j) 
    Psi_j = 1/(2j) * (Y_j_H - Y_j)

    Phi.append(Phi_j.real)  # We could keep them complex, but we make real for now
    Psi.append(Psi_j.real)

# # Objective function: Minimize Pg, the matrix representing the real power generation at each generator 
# Pg = cp.real(cp.trace(W @ Ybus))  # P_g = XY. The trace sums up all the power generation and estimate the cost. We also ensure Pg is real-valued
# c2 = np.array([0.1, 0.2])  # Quadratic cost coefficients
# c1 = np.array([10, 20])  # Linear cost coefficients
# objective = cp.Minimize(cp.sum(c2 * Pg**2 + c1 * Pg)) #Quadratic Objective function

# Objective Function
objective_terms = [
                   c1[i] * cp.real(cp.trace(Ybus[int(bus), :] @ W @ Ybus[:, int(bus)])) +
                   c0[i] for i, bus in enumerate(gen_buses)]
objective = cp.Minimize(cp.sum(objective_terms))

# Constraints
constraints = []
# Positive semidefinite constraint
constraints.append(W >> 0)
# constraints.append(W[slack_bus, slack_bus] == 1)  # Slack bus voltage is 1


for j in range(n):
    # Pin_j = cp.real(Ybus[i, :] @ W @ cp.conj(Ybus[:,i]))  # Extract real power injection at bus i
    # Qin_j = cp.imag(Ybus[i, :] @ W @ cp.conj(Ybus[:,i])) # Extract reactive power injection at bus i

    #Voltage magnitude constraints
    # if j!= slack_bus:
    constraints.append(V_min[j]<=cp.real(W[j,j]))
    constraints.append(cp.real(W[j,j])<=V_max[j])

    # Power balance equations (real and reactive)
    Pin_j = cp.trace(Phi[j]@W)
    Qin_j = cp.trace(Psi[j]@W)

    # If bus is a generator bus, enforce generation limits
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        constraints.append(P_min_gen[idx] <= cp.real(Pin_j))
        constraints.append(cp.real(Pin_j) <= P_max_gen[idx])
        constraints.append(Q_min_gen[idx] <= cp.real(Qin_j))
        constraints.append(cp.real(Qin_j) <= Q_max_gen[idx])
    else:
        # For non-generator buses, enforce power balance constraints
        constraints.append(Pin_j == p_load[j])
        constraints.append(Qin_j == q_load[j])


# # Voltage magnitude constraints
# for i in range(n):
#     constraints.append(V_min[i] <= cp.real(W[i, i]))
#     constraints.append(cp.real(W[i, i]) <= V_max[i])



In [39]:
for j in range(n):
    print(Qin_j)


N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A


### Solve using cvxpy's SDP solver

In [34]:
# Solve the SDP
print("P_min:", P_min_gen)
print("P_max:", P_max_gen)
print("Q_min:", Q_min_gen)
print("Q_max:", Q_max_gen)

# prob = cp.Problem(objective, constraints)
prob = cp.Problem(cp.Maximize(0), constraints)
# print("Pg Limits:", Pg_min_gen, Pg_max_gen)
# print("Voltage Limits:", V_min, V_max)
# print("Power Demand:", p_load, q_load)
prob.solve(solver=cp.MOSEK, verbose=True)

# Print results
print("Optimal cost:", prob.value)
print("Optimal voltage matrix X:\n", W.value)

P_min: [0. 0. 0. 0. 0.]
P_max: [80. 50. 55. 30. 40.]
Q_min: [-20. -15. -15. -10. -15.]
Q_max: [60.  62.5 48.7 40.  44.7]
                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Mar 11 12:50:18 PM: Your problem has 900 variables, 1030 constraints, and 0 parameters.
(CVXPY) Mar 11 12:50:18 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 11 12:50:18 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 11 12:50:18 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 11 12:50:18 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
---------------------------

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


In [23]:
# Check problem status
if prob.status == cp.OPTIMAL:
    print("Optimal cost:", prob.value)
    print("Optimal voltage matrix X:\n", W.value)
    print("Optimal real power demand Pd:\n", Pd.value)
    print("Optimal reactive power demand Qd:\n", Qd.value)
elif prob.status == cp.INFEASIBLE:
    print("The problem is infeasible.")
elif prob.status == cp.UNBOUNDED:
    print("The problem is unbounded.")
else:
    print("Solver status:", prob.status)

The problem is infeasible.
